In [ ]:
# 这个代码文件是把一个预训练模型ERNIE1.0构成的神经网络 利用in-batch-negative方法进行训练

In [1]:
import abc
import sys
import argparse
import os
import random
import time
import numpy as np
from scipy import stats
import pandas as pd
from tqdm import tqdm 
from scipy.special import softmax
from scipy.special import expit
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle import inference
import paddlenlp
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset, MapDataset
from paddlenlp.transformers import LinearDecayWithWarmup
from paddlenlp.utils.downloader import get_path_from_url
from visualdl import LogWriter
from data import convert_pairwise_example

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [3]:
paddle.set_device("gpu")

Place(gpu:0)

In [4]:
def read_text_pair(data_path): 
  with open(data_path, 'r', encoding='utf-8') as f: 
        for line in f: 
            data = line.rstrip().split("\t") 
            if len(data) != 2: 
                continue
            yield {'text_a': data[0], 'text_b': data[1]} 
                                                         


In [5]:

train_set_path='recall_dataset/train.csv' 
train_ds = load_dataset(read_text_pair, data_path=train_set_path, lazy=False) 

In [6]:
print(type(train_ds))

<class 'paddlenlp.datasets.dataset.MapDataset'>


In [7]:

for i in range(3):
    print(train_ds[i])

{'text_a': '从《唐律疏义》看唐代封爵贵族的法律特权', 'text_b': '从《唐律疏义》看唐代封爵贵族的法律特权《唐律疏义》,封爵贵族,法律特权'}
{'text_a': '宁夏社区图书馆服务体系布局现状分析', 'text_b': '宁夏社区图书馆服务体系布局现状分析社区图书馆,社区图书馆服务,社区图书馆服务体系'}
{'text_a': '人口老龄化对京津冀经济', 'text_b': '京津冀人口老龄化对区域经济增长的影响京津冀,人口老龄化,区域经济增长,固定效应模型'}


In [ ]:
# 下面开始构造训练数据的加载器

In [8]:
MODEL_NAME="ernie-1.0"

tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

[2025-02-06 22:10:15,430] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie/vocab.txt and saved to C:\Users\LJX\.paddlenlp\models\ernie-1.0
[2025-02-06 22:10:15,973] [    INFO] - Downloading vocab.txt from https://bj.bcebos.com/paddlenlp/models/transformers/ernie/vocab.txt
100%|██████████| 89.5k/89.5k [00:00<00:00, 98.2kB/s]
[2025-02-06 22:10:17,458] [    INFO] - tokenizer config file saved in C:\Users\LJX\.paddlenlp\models\ernie-1.0\tokenizer_config.json
[2025-02-06 22:10:17,461] [    INFO] - Special tokens file saved in C:\Users\LJX\.paddlenlp\models\ernie-1.0\special_tokens_map.json


In [9]:
def convert_example(example, tokenizer, max_seq_length=512): 
    result = []
    
    for key, text in example.items(): 
        encoded_inputs = tokenizer(text=text, max_seq_len=max_seq_length)
        input_ids = encoded_inputs["input_ids"] 
        token_type_ids = encoded_inputs["token_type_ids"] 
        result += [input_ids, token_type_ids]

    return result 

In [10]:
#演示

a=[]
for key, text in train_ds[0].items():
    print(text)
    encoded_inputs = tokenizer(text=text, max_seq_len=512)
    print(encoded_inputs)
    print(encoded_inputs["input_ids"])
    print(encoded_inputs["token_type_ids"])
    a += [encoded_inputs["input_ids"], encoded_inputs["token_type_ids"]]
    print('------------------------------------')
    
print(a)

从《唐律疏义》看唐代封爵贵族的法律特权
{'input_ids': [1, 158, 56, 867, 646, 1500, 393, 55, 335, 867, 140, 898, 2153, 864, 495, 5, 72, 646, 169, 438, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
[1, 158, 56, 867, 646, 1500, 393, 55, 335, 867, 140, 898, 2153, 864, 495, 5, 72, 646, 169, 438, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------------
从《唐律疏义》看唐代封爵贵族的法律特权《唐律疏义》,封爵贵族,法律特权
{'input_ids': [1, 158, 56, 867, 646, 1500, 393, 55, 335, 867, 140, 898, 2153, 864, 495, 5, 72, 646, 169, 438, 56, 867, 646, 1500, 393, 55, 30, 898, 2153, 864, 495, 30, 72, 646, 169, 438, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
[1, 158, 56, 867, 646, 1500, 393, 55, 335, 867, 140, 898, 2153, 864, 495, 5, 72, 646, 169, 438, 56, 867, 646, 1500, 393, 55, 30, 898, 2153, 864, 495, 30, 72, 646, 169, 438, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2293: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1865: UserWarning: Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
  warnings.warn(


In [11]:
from functools import partial

trans_func = partial(convert_example, tokenizer=tokenizer, max_seq_length=64) 

In [12]:
def batchify_fn(samples):
    fn = Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),   
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype='int64'),  
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype='int64'),  
    )

    processed_samples = fn(samples) 

    result = []
    for data in processed_samples:
        result.append(data) 

    return result 

In [13]:
batch_sampler = paddle.io.DistributedBatchSampler(train_ds, batch_size=64, shuffle=True)
train_data_loader = paddle.io.DataLoader(dataset=train_ds.map(trans_func), batch_sampler=batch_sampler, collate_fn=batchify_fn, return_list=True)

In [ ]:
# 下面开始搭建召回模型

In [14]:
pretrained_model = paddlenlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)

[2025-02-06 22:16:54,773] [    INFO] - Configuration saved in C:\Users\LJX\.paddlenlp\models\ernie-1.0\config.json
[2025-02-06 22:16:55,190] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie/ernie_v1_chn_base.pdparams
[2025-02-06 22:16:55,193] [    INFO] - Downloading ernie_v1_chn_base.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie/ernie_v1_chn_base.pdparams
100%|██████████| 383M/383M [08:34<00:00, 781kB/s]    
[2025-02-06 22:25:30,563] [    INFO] - Loading weights file model_state.pdparams from cache at C:\Users\LJX\.paddlenlp\models\ernie-1.0\model_state.pdparams
[2025-02-06 22:25:30,834] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-02-06 22:25:31,928] [ WARNING] - Some weights of the model checkpoint at ernie-1.0 were not used when initializing ErnieModel: ['cls.predictions.transform.bias', 'cls.predictions.decoder_bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'c

In [15]:
from base_model import SemanticIndexBase

class SemanticIndexBatchNeg(SemanticIndexBase): 
    def __init__(self, pretrained_model, dropout=None, margin=0.3, scale=30, output_emb_size=None):
        super().__init__(pretrained_model, dropout, output_emb_size)

        self.margin = margin
        self.scale = scale

    def forward(self, query_input_ids,    title_input_ids,    query_token_type_ids=None, query_position_ids=None, query_attention_mask=None,    title_token_type_ids=None, title_position_ids=None, title_attention_mask=None):
        query_cls_embedding = self.get_pooled_embedding(query_input_ids, query_token_type_ids, query_position_ids, query_attention_mask) 

        title_cls_embedding = self.get_pooled_embedding(title_input_ids, title_token_type_ids, title_position_ids, title_attention_mask)    
        
        cosine_sim = paddle.matmul(query_cls_embedding, title_cls_embedding, transpose_y=True)  
        
        margin_diag = paddle.full(shape=[query_cls_embedding.shape[0]], fill_value=self.margin, dtype="float32") 

        cosine_sim = cosine_sim - paddle.diag(margin_diag)

        cosine_sim = cosine_sim * self.scale

        labels = paddle.arange(0, query_cls_embedding.shape[0], dtype='int64') 
        labels = paddle.reshape(labels, shape=[-1, 1]) 

        loss = F.cross_entropy(input=cosine_sim, label=labels)

        return loss

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddle\jit\dy2static\program_translator.py:712: UserWarning: full_graph=False don't support input_spec arguments. It will not produce any effect.
You can set full_graph=True, then you can assign input spec.

  warnings.warn(


In [16]:
model = SemanticIndexBatchNeg(pretrained_model, margin=0.1, scale=20, output_emb_size=256)

In [ ]:
# 下面开始定义模型训练用到的各种参数，并进行模型训练

In [17]:
epochs=3 
num_training_steps = len(train_data_loader) * epochs 

In [18]:
lr_scheduler = LinearDecayWithWarmup(5E-5, num_training_steps, 0.0) 

In [19]:
decay_params = [
        p.name for n, p in model.named_parameters() 
        if not any(nd in n for nd in ["bias", "norm"]) 
    ] 

In [20]:
optimizer = paddle.optimizer.AdamW( 
    learning_rate=lr_scheduler, 
    parameters=model.parameters(), 
    weight_decay=0.0, 
    apply_decay_param_fun=lambda x: x in decay_params)

In [21]:
save_dir='model_param'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

global_step = 0 
tic_train = time.time()

for epoch in range(1, epochs + 1): 
    for step, batch in enumerate(train_data_loader, start=1): 
        query_input_ids, query_token_type_ids, title_input_ids, title_token_type_ids = batch 

        loss = model(query_input_ids=query_input_ids, title_input_ids=title_input_ids, query_token_type_ids=query_token_type_ids, title_token_type_ids=title_token_type_ids)

        global_step += 1 
        if global_step % 10 == 0: 
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, 10 / (time.time() - tic_train))) 
            tic_train = time.time() 

        loss.backward() 
        optimizer.step() 
        lr_scheduler.step() 
        optimizer.clear_grad() 

        if global_step % 10 == 0: 
            save_path = os.path.join(save_dir, "model_%d" % global_step)
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            save_param_path = os.path.join(save_path, 'model_state.pdparams') 
            paddle.save(model.state_dict(), save_param_path) 
            tokenizer.save_pretrained(save_path) 

c:\Users\LJX\Anaconda3\envs\search\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2293: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(


global step 10, epoch: 1, batch: 10, loss: 2.26462, speed: 0.65 step/s


[2025-02-06 22:32:06,411] [    INFO] - tokenizer config file saved in model_param\model_10\tokenizer_config.json
[2025-02-06 22:32:06,414] [    INFO] - Special tokens file saved in model_param\model_10\special_tokens_map.json


global step 20, epoch: 1, batch: 20, loss: 1.28187, speed: 0.54 step/s


[2025-02-06 22:32:24,095] [    INFO] - tokenizer config file saved in model_param\model_20\tokenizer_config.json
[2025-02-06 22:32:24,098] [    INFO] - Special tokens file saved in model_param\model_20\special_tokens_map.json


...

[2025-02-06 22:36:27,100] [    INFO] - tokenizer config file saved in model_param\model_160\tokenizer_config.json
[2025-02-06 22:36:27,102] [    INFO] - Special tokens file saved in model_param\model_160\special_tokens_map.json


global step 170, epoch: 3, batch: 44, loss: 0.33112, speed: 0.71 step/s


[2025-02-06 22:36:41,463] [    INFO] - tokenizer config file saved in model_param\model_170\tokenizer_config.json
[2025-02-06 22:36:41,465] [    INFO] - Special tokens file saved in model_param\model_170\special_tokens_map.json


global step 180, epoch: 3, batch: 54, loss: 0.32440, speed: 0.64 step/s


[2025-02-06 22:36:57,125] [    INFO] - tokenizer config file saved in model_param\model_180\tokenizer_config.json
[2025-02-06 22:36:57,127] [    INFO] - Special tokens file saved in model_param\model_180\special_tokens_map.json
